# Ralphy Núñez Mercado

## T2.3: Scrapear personaxes e capítulos de Rick e Morty

Scraping nivel 0. Servizos web
Apuntes relacionados: https://github.com/jfsanchez/SBD/blob/main/notebooks/scraping/scraping-0-API-REST.ipynb

Exercicio: Das URL seguintes podes obter os episodios e personaxes.

Capítulos: https://rickandmortyapi.com/api/episode
Personaxes: https://rickandmortyapi.com/api/character
Queremos saber por unha banda as personaxes e por outra os capítulos.

Coidado! A API ten un paxinador:

https://rickandmortyapi.com/api/character?page=42
https://rickandmortyapi.com/api/episode?page=3
Tes que recorrer as páxinas ata que en info, next sexa null.

O código debe ser xenérico e ser chamado cunha función tanto para personaxes como para episodios.

Deben gardarse os datos nunha base de datos de PostgreSQL.

### Imports necesarios

In [1]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import json

## Creando a cadea de conexión

In [ ]:
#Datos de conexión
db_host = "10.133.28.46" # ip cesga
db_port=5432 
db_user = "postgres"
db_passwd="Cl431Ns3gur4"
db_name="rickymorty"



## Recorrer la API

In [ ]:
URL_episodio = "https://rickandmortyapi.com/api/episode"
URL_personaje = "https://rickandmortyapi.com/api/character" 

def recorrer(URL):
    paginas = requests.get(URL).json()["info"]["pages"] 
    resultados = []

    for i in range(1, paginas + 1):
        print(f"Recorriendo página {i}")
        response = requests.get(f"{URL}?page={i}").json()
        resultados.extend(response["results"])
    
    return resultados  

episodios = recorrer(URL_episodio)
print(f"Episodios : {len(episodios)}")


Recorriendo página 1
Recorriendo página 2
Recorriendo página 3
Episodios : 51


[{'id': 1,
  'name': 'Pilot',
  'air_date': 'December 2, 2013',
  'episode': 'S01E01',
  'characters': ['https://rickandmortyapi.com/api/character/1',
   'https://rickandmortyapi.com/api/character/2',
   'https://rickandmortyapi.com/api/character/35',
   'https://rickandmortyapi.com/api/character/38',
   'https://rickandmortyapi.com/api/character/62',
   'https://rickandmortyapi.com/api/character/92',
   'https://rickandmortyapi.com/api/character/127',
   'https://rickandmortyapi.com/api/character/144',
   'https://rickandmortyapi.com/api/character/158',
   'https://rickandmortyapi.com/api/character/175',
   'https://rickandmortyapi.com/api/character/179',
   'https://rickandmortyapi.com/api/character/181',
   'https://rickandmortyapi.com/api/character/239',
   'https://rickandmortyapi.com/api/character/249',
   'https://rickandmortyapi.com/api/character/271',
   'https://rickandmortyapi.com/api/character/338',
   'https://rickandmortyapi.com/api/character/394',
   'https://rickandmort

(en los episodio guardar  los personajes pero en los personajes no guardar los episodios)

## Creacion de la base de datos postgresql rick y morty

### Primero crearemos el Docker   




```bash
docker run 
\ --name de-postre-sql 
\ -v /home/cesgaxuser/BBDD:/BBDDdocker 
\ -e POSTGRES_PASSWORD=Cl431Ns3gur4 
\ --restart unless-stopped 
\ -p 5432:5432 
\ -p 5433:5433 
\ -d postgres
```


### Entramos en el docker y creamos la base de datos  

```bash
docker exec -it de-postre-sql psql -U postgres
```

### Creación de las tablas para nuestra BD

Episodio:

```sql
CREATE TABLE episodio (id INT, 
    name VARCHAR(255), 
    air_date VARCHAR(255), 
    episode VARCHAR(255), 
    characters text[], 
    url text, 
    created text);
```
Personaje: 

```sql
CREATE TABLE personaje (id INT,   
    name VARCHAR(255), 
    status VARCHAR(255), 
    species VARCHAR(255), 
    type VARCHAR(255), 
    gender VARCHAR(255), 
    origin TEXT, 
    location TEXT, 
    image text, 
    url text, 
    created text);
```

### Ahora tendremos que insertar los datos de la API a las tablas de nuestra base de datos de PostgreSQL ( OJO que en la tabla personaje no vamos a guardar los episodios )

### Primero vamos a probar la conexion con la base de datos en postgreSQL

Anteriormente usabamos una conection string como esta:
```python 
    connectionString=f'mysql+pymysql://{db_user}:{db_passwd}@{db_host}:{db_port}/{db_name}'
```
El problema es que esta connectionString es para mysql y en este ejercicio usamos postgreSQL.  

Para poder conectarnos a una BD en postgreSQL tendremos que instalar esta librería: 

In [ ]:
%pip install psycopg2 
# %conda install anaconda::psycopg2

Con la libreria instalada esta sería la connectionString adecuada para conectarnos.

In [9]:
connectionString = f'postgresql+psycopg2://{db_user}:{db_passwd}@{db_host}:{db_port}/{db_name}'

In [89]:
from sqlalchemy import create_engine, text
from sqlalchemy import insert

engine = create_engine(connectionString)

cadeaSQL = "SELECT * FROM episodio"
dfPanda = pd.read_sql(cadeaSQL, engine)
print(dfPanda.head())


   id                    name           air_date episode  \
0   1                   Pilot   December 2, 2013  S01E01   
1   2           Lawnmower Dog   December 9, 2013  S01E02   
2   3            Anatomy Park  December 16, 2013  S01E03   
3   4  M. Night Shaym-Aliens!   January 13, 2014  S01E04   
4   5    Meeseeks and Destroy   January 20, 2014  S01E05   

                                          characters  \
0  [https://rickandmortyapi.com/api/character/1, ...   
1  [https://rickandmortyapi.com/api/character/1, ...   
2  [https://rickandmortyapi.com/api/character/1, ...   
3  [https://rickandmortyapi.com/api/character/1, ...   
4  [https://rickandmortyapi.com/api/character/1, ...   

                                         url                   created  
0  https://rickandmortyapi.com/api/episode/1  2017-11-10T12:56:33.798Z  
1  https://rickandmortyapi.com/api/episode/2  2017-11-10T12:56:33.916Z  
2  https://rickandmortyapi.com/api/episode/3  2017-11-10T12:56:34.022Z  
3  https:/

Ahora ya podremos insertar los datos en las tablas.

In [ ]:
json_episodios = recorrer(URL_episodio)

with engine.connect() as conn:
    for episodio in json_episodios:
        id = episodio['id']
        name = episodio['name'].replace("'","")
        air_date = episodio['air_date']
        episode = episodio['episode']
        characters = episodio['characters']
        url = episodio['url']
        created = episodio['created']
        
        sql = f"""
            INSERT INTO episodio ( id, name, air_date, episode, characters, url, created) 
            VALUES ({id}, '{name}', '{air_date}', '{episode}', array{str(characters)}, '{url}', '{created}')
        """

        conn.execute(text(sql))
    conn.commit()



Recorriendo página 1
Recorriendo página 2
Recorriendo página 3
Inserting episodio: {'id': 1, 'name': 'Pilot', 'air_date': 'December 2, 2013', 'episode': 'S01E01', 'characters': ['https://rickandmortyapi.com/api/character/1', 'https://rickandmortyapi.com/api/character/2', 'https://rickandmortyapi.com/api/character/35', 'https://rickandmortyapi.com/api/character/38', 'https://rickandmortyapi.com/api/character/62', 'https://rickandmortyapi.com/api/character/92', 'https://rickandmortyapi.com/api/character/127', 'https://rickandmortyapi.com/api/character/144', 'https://rickandmortyapi.com/api/character/158', 'https://rickandmortyapi.com/api/character/175', 'https://rickandmortyapi.com/api/character/179', 'https://rickandmortyapi.com/api/character/181', 'https://rickandmortyapi.com/api/character/239', 'https://rickandmortyapi.com/api/character/249', 'https://rickandmortyapi.com/api/character/271', 'https://rickandmortyapi.com/api/character/338', 'https://rickandmortyapi.com/api/character/394

In [91]:

count = "SELECT count(*) FROM episodio "
dfCount = pd.read_sql(count, engine)
print(dfCount)
dfPanda

   count
0     51


,id,name,air_date,episode,characters,url,created
0,1,Pilot,"December 2, 2013",S01E01,"[https://rickandmortyapi.com/api/character/1, ...",https://rickandmortyapi.com/api/episode/1,2017-11-10T12:56:33.798Z
1,2,Lawnmower Dog,"December 9, 2013",S01E02,"[https://rickandmortyapi.com/api/character/1, ...",https://rickandmortyapi.com/api/episode/2,2017-11-10T12:56:33.916Z
2,3,Anatomy Park,"December 16, 2013",S01E03,"[https://rickandmortyapi.com/api/character/1, ...",https://rickandmortyapi.com/api/episode/3,2017-11-10T12:56:34.022Z
3,4,M. Night Shaym-Aliens!,"January 13, 2014",S01E04,"[https://rickandmortyapi.com/api/character/1, ...",https://rickandmortyapi.com/api/episode/4,2017-11-10T12:56:34.129Z
4,5,Meeseeks and Destroy,"January 20, 2014",S01E05,"[https://rickandmortyapi.com/api/character/1, ...",https://rickandmortyapi.com/api/episode/5,2017-11-10T12:56:34.236Z
5,6,Rick Potion #9,"January 27, 2014",S01E06,"[https://rickandmortyapi.com/api/character/1, ...",https://rickandmortyapi.com/api/episode/6,2017-11-10T12:56:34.339Z
6,7,Raising Gazorpazorp,"March 10, 2014",S01E07,"[https://rickandmortyapi.com/api/character/1, ...",https://rickandmortyapi.com/api/episode/7,2017-11-10T12:56:34.441Z
7,8,Rixty Minutes,"March 17, 2014",S01E08,"[https://rickandmortyapi.com/api/character/1, ...",https://rickandmortyapi.com/api/episode/8,2017-11-10T12:56:34.543Z
8,9,Something Ricked This Way Comes,"March 24, 2014",S01E09,"[https://rickandmortyapi.com/api/character/1, ...",https://rickandmortyapi.com/api/episode/9,2017-11-10T12:56:34.645Z
9,10,Close Rick-counters of the Rick Kind,"April 7, 2014",S01E10,"[https://rickandmortyapi.com/api/character/1, ...",https://rickandmortyapi.com/api/episode/10,2017-11-10T12:56:34.747Z


Total de episodios en la base de datos: 51
